In [1]:
import os, sys
sys.path.insert(0, "\\".join(os.path.abspath(os.curdir).split("\\")[:-2:]))
from pathlib import Path

import matplotlib.pyplot as plt
from pydicom import dcmread
import pandas as pd
import numpy as np
import json

from src.utils.utils import save_json, get_dicom_meta, preprocessing_path

In [3]:
def update_count_tag(list_tags: list | set, dictionary_tags: dict) -> None:
    for key in list_tags:
        if key in dictionary_tags.keys():
            dictionary_tags[key] += 1
        else:
            dictionary_tags[key] = 1

metadata_files = ['calc_case_description_test_set',
             'mass_case_description_train_set',
             'calc_case_description_train_set',
             'mass_case_description_test_set']

dictionary_metadata = {}
dictionary_empty_meta = {}

for current_meta in metadata_files:
    df = pd.read_csv(f"../../metadata//{current_meta}.csv").iloc[:5:, ::]
    
    for i in range(df.shape[0]):
        original_image_path = df.iloc[i][11]
        original_image_path = preprocessing_path(original_image_path)
        
        cropped_images_path = df.iloc[i][12]
        cropped_images_path = preprocessing_path(cropped_images_path)
        
        all_image_path = {"original": original_image_path, "cropped":cropped_images_path}
        dicom_files = []
        
        for key, path in all_image_path.items():
            path_exam_files = f"D:/mathe/Documents/BancoDados_IC/CBIS-DDSM/{path}/"
            directory = Path(path_exam_files)
            paths_dicom_images = list(directory.rglob("*.dcm*"))
            
            for path_dicom in paths_dicom_images:
                dicom_file = dcmread(path_dicom)
                dicom_files.append(get_dicom_meta(dicom_file))
                
        all_tags = set()
        empty_tags = set()
        
        for dicom_file in dicom_files:
            all_tags = all_tags | set(dicom_file.keys()) # união
            empty_tags = empty_tags | set([key for key, value in dicom_file.items() if dicom_file[key] == ""]) # união
            
        update_count_tag(all_tags, dictionary_metadata)
        update_count_tag(empty_tags, dictionary_empty_meta)
        
dictionary = {"all": dictionary_metadata, "empty": dictionary_empty_meta}
#save_json("dictionary_metadata", dictionary, f"../../metadata/dictionary_metadata")  
dictionary 

{'all': {'study_id (0020, 0010)': 20,
  'pixel_representation (0028, 0103)': 20,
  'smallest_image_pixel_value (0028, 0106)': 20,
  'study_date (0008, 0020)': 20,
  'patient_id (0010, 0020)': 20,
  'instance_number (0020, 0013)': 20,
  'content_time (0008, 0033)': 20,
  'study_time (0008, 0030)': 20,
  'content_date (0008, 0023)': 20,
  'modality (0008, 0060)': 20,
  'photometric_interpretation (0028, 0004)': 20,
  "patient's_sex (0010, 0040)": 20,
  'private_tag_data (0013, 1010)': 20,
  'sop_class_uid (0008, 0016)': 20,
  'bits_allocated (0028, 0100)': 20,
  'series_description (0008, 103e)': 20,
  'private_creator (0013, 0010)': 20,
  'sop_instance_uid (0008, 0018)': 20,
  'samples_per_pixel (0028, 0002)': 20,
  'high_bit (0028, 0102)': 20,
  'conversion_type (0008, 0064)': 20,
  "patient's_name (0010, 0010)": 20,
  'series_number (0020, 0011)': 20,
  'accession_number (0008, 0050)': 20,
  'series_instance_uid (0020, 000e)': 20,
  'study_instance_uid (0020, 000d)': 20,
  'specific_c